# 1. Qualifying set

# 1.1 Convert hex to base64

In [1]:
import codecs
a = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
b64 = codecs.encode(codecs.decode(a, 'hex'), 'base64').decode()
print(b64)

SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t



# 1.2 Fixed XOR

In [2]:
import binascii
def fixedxor(str1, str2):
    if len(str1) == len(str2) :     
        A = binascii.a2b_hex(str1)
        B = binascii.a2b_hex(str2)
        xor_result = binascii.b2a_hex(bytes([ x^y for (x,y) in zip(A, B)]))
        return xor_result
    else :
        print('les deux strings sont de longeur différente.')

In [3]:
a = '1c0111001f010100061a024b53535009181c'
b = '686974207468652062756c6c277320657965'
xor_result = fixedxor(a,b)
print(xor_result)

b'746865206b696420646f6e277420706c6179'


# 1.3 Single-byte XOR cipher

In [4]:
import binascii
encode = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
def single_byte_xor(encode) :
    ciphertext = binascii.a2b_hex(encode)
    for key in range(256) :
        result = (''.join(chr(num ^ key) for num in ciphertext) for key in range(256))
    return max(result, key=lambda s: s.count(' '))

In [5]:
result = single_byte_xor(encode)
print(result)

Cooking MC's like a pound of bacon


# Challenge 4 - Detect single-character XOR

In [6]:
with open('./detecting-singlechar-xor.txt') as data_file:
    ciphertext_list = [
        binascii.a2b_hex(line.strip())
        for line in data_file
    ]

In [7]:
def detect_single_character_xor(ciphertext):
    best = {"nombre_lettere": 0}
    for i in range(256) :   
        key_value = i.to_bytes(1, byteorder='big') *len(ciphertext)
        char_text = list(range(97, 122))
        char_text.append(32)
        message = bytes([ x^y for (x,y) in zip(ciphertext, key_value)])
        nb_letters = sum([ x in char_text for x in message])
        if nb_letters>best['nombre_lettere']:
            best = {"message": message, 'nombre_lettere': nb_letters, 'key': i.to_bytes(1, byteorder='big')}
    
    if best['nombre_lettere'] > 0.7*len(ciphertext):
        return best
    else:
        pass

In [8]:
resultat = list()
for (line_nb, ciphertext) in enumerate(ciphertext_list):
    try:
        message = detect_single_character_xor(ciphertext)['message']
    except :
        pass
    else:
        resultat.append({
            'line_nb': line_nb,
            'ciphertext': ciphertext,
            'message': message
        })
        
if len(resultat) > 1:
    print("Error: plus qu'un seul candidat")
else:
    for (key, value) in resultat[0].items():
        print(key,':',value)

line_nb : 170
ciphertext : b'{ZB\x15A]TA\x15A]P\x15ETGAL\x15\\F\x15_@XE\\[R?'
message : b'Now that the party is jumping\n'


# 1.5 Implement repeating-key XOR

In [9]:
message = b"Bonjour, vous allez bien ?" 
key = b'ICE'
streamkey = key*(len(message)//len(key) + 1)
ciphertext = bytes([ x^y for (x,y) in zip(message, streamkey)])
expected_result = binascii.b2a_hex(ciphertext)

In [10]:
print(expected_result)

b'0b2c2b232c303b6f653f2c303a6324252f2033632720262b697c'


# 1.6 Break repeating-key XOR

In [11]:
from base64 import b64decode
with open("./repeating-xor.txt") as file:
        ciphertext = b64decode(file.read())

In [12]:
def hamming_distance(a, b):
    byte_xor = bytes([ x^y for (x,y) in zip(a, b)])
    result = sum(bin(byte).count('1') for byte in byte_xor)
    return result

In [13]:
a=  b"this is a test"
b = b"wokka wokka !!!"
result = hamming_distance(a, b)
print('haming distance est :', result)

haming distance est : 38


In [14]:
key = lambda x: distance_by_key_size(x,ciphertext)
print(key)

<function <lambda> at 0x7fe6201aeb70>


In [15]:
def distance_by_key_size(key_size, ciphertext):
    size = 2*key_size
    nb_measurements = len(ciphertext) // size - 1
    score = 0
    for i in range(0,nb_measurements):
        z = size
        k = key_size
        size1 = slice(i*z, i*z + k)
        size2 = slice(i*z + k, i*z + 2*k)
        score += hamming_distance(ciphertext[size1], ciphertext[size2])
    score /= key_size  
    score /= nb_measurements
    return score

In [16]:
def find_by_key_length(ciphertext, min_length, max_length):
    
    key = lambda x: distance_by_key_size(x,ciphertext)
    result = min(range(min_length, max_length), key=key)
    return result

In [17]:
min_length = 2
max_length = 50
def attack_repeating_key_xor(ciphertext,min_length,max_length):
    keysize = find_by_key_length(ciphertext,min_length,max_length)
    key = bytes()
    message_parts = list()
    for i in range(keysize):
        part = detect_single_character_xor(bytes(ciphertext[i::keysize]))
        key += part["key"]
        message_parts.append(part["message"])
    message = bytes()
    for i in range(max(map(len, message_parts))):
        message += bytes([part[i] for part in message_parts if len(part)>=i+1])
    result = {'message':message, 'key':key}
    return result         

In [18]:
result = attack_repeating_key_xor(ciphertext,min_length,max_length)
print("la clef est:",result["key"],'\n')
print('le message est :\n')
print(result["message"].decode())

la clef est: b'Terminator X: Bring the noise' 

le message est :

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effe

# 1.7 AES in ECB mode

In [46]:
from base64 import b64decode
with open("./aes-in-ecb.txt") as file:
    ciphertext = b64decode(file.read())
key=b"YELLOW SUBMARINE"

In [47]:
from Crypto.Cipher import AES
def decrypt_aes_128_ecb(ctxt, key):
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(ctxt) 
    return plaintext

In [48]:
plaintext = decrypt_aes_128_ecb(ciphertext,key)
print(plaintext)

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

# 1.8 Detect AES in ECB mode

In [22]:
with open('./detect-aes-ecb.txt') as f:
    ctxts = [binascii.a2b_hex(line.strip()) for line in f]
blocksize = 16    

In [23]:
def detect_repeated_blocks(ctxt, blocksize):
    if len(ctxt) % blocksize != 0:
        print('la longueur du texte chiffré n est pas un multiple de la taille du bloc')
    else:
        num_blocks = len(ctxt) // blocksize    
    blocks = [ctxt[i*blocksize:(i+1)*blocksize] for i in range(num_blocks)] 
    if len(set(blocks)) - num_blocks != 0:
        return True
    else:
        return False

In [24]:
result = [ctxt for ctxt in ctxts if detect_repeated_blocks(ctxt,blocksize)]
print('le bloc répété est:\n', result)

le bloc répété est:
 [b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j']


# 2. Block crypto

# 2.1 Implement PKCS#7 padding

In [25]:
import random
from random import randint
import os
def pkcs7_padding(message, block_size):
    padding_length = block_size - (len(message) % block_size )
    if padding_length == 0:
        padding_length = block_size
    padding = bytes([padding_length]) * padding_length
    result = message + padding
    return result

In [26]:
message = b'YELLOW SUBMARINE'
block_size = 20
pkcs7_padding(message, block_size)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

# 2.2 Implement CBC mode

In [27]:
from base64 import b64encode
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import unpad

with open("./cbc-mode.txt") as file:
    ctxts = [b64decode(line.strip()) for line in file]
key = get_random_bytes(16)

def encrypt_aes_128_cbc(ctxts, key) :
    for i in range(len(ctxts)) :
        cipher = AES.new(key, AES.MODE_CBC)
        ct_bytes = cipher.encrypt(pad(ctxts[i], AES.block_size))
        iv = b64encode(cipher.iv).decode('utf-8')
        ct = b64encode(ct_bytes).decode('utf-8')
    result = {'iv':iv, 'ciphertext':ct}
    return result


def decrypt_aes_128_cbc(result, key) :
    try:
        iv = b64decode(result['iv'])
        ct = b64decode(result['ciphertext'])
        cipher = AES.new(key, AES.MODE_CBC, iv)     
        pt = pkcs7_padding(cipher.decrypt(ct), AES.block_size)
        print("Le message décrypté est :\n : ", pt)
    except ValueError :     
        print("decryption incorrecte")

In [28]:
ctxt = encrypt_aes_128_cbc(ctxts, key)
print('le message encrypté est :\n',ctxt)
msg = decrypt_aes_128_cbc(ctxt, key)
print(msg)

le message encrypté est :
 {'iv': 'fJKTDrYWwmo4bLXZ8pCURg==', 'ciphertext': 'wJme3VCqoh4+0NIiJvdKxiT8Xq7aTMhRd38ZG3a0tAsV1WP6tY7NiaWbm9GQ9oeq'}
Le message décrypté est :
 :  b"\x92B3Z\x149G\x17\x7f\xbes6A\n\xfd\x0b\x16\xb6'\xd1\x97\x11]\xe4\x188\x99W\xdd>< \xd2Tf8V\x01|\xcb\xb1\x9et\x8da"
None


# 2.3 An ECB/CBC detection oracle

In [30]:
from Crypto.Cipher import AES


def encrypt_aes_128_cbc(ctxts, key) :
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(ctxts, AES.block_size))
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    result = {'iv':iv, 'ciphertext':ct}
    return result

def encrypt_aes_128_ecb(ctxt, key):
    block_size = 16  # Bytes
    raw = pkcs7_padding(message, block_size)
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.encrypt(raw) 
    return plaintext

def encrypt_oracle(message):
    key = os.urandom(16)
    left = os.urandom(randint(5, 10))
    right = os.urandom(randint(5, 10))
    iv=os.urandom(16)
    message = left + message + left
    if randint(0, 1) == 0:
        return encrypt_aes_128_ecb(message, key)
    else:
        return encrypt_aes_128_cbc(message, key)

from functools import reduce as rd
def detect_ecb(plaintext, ciphertext):
    blocks = [
        ciphertext[i:i+16] \
           for i in range(0, len(ciphertext) - 16, 16)
    ]
    increment = lambda d, b: \
       (b in d and (d.__setitem__(b, d[b] + 1) or d)) or \
          (d.__setitem__(b, 1) or d)
    items = list(rd(increment, blocks, {}).items())
    items.sort(key=lambda p: p[1], reverse=True)
    if len(plaintext) // 16 - items[0][1] < 2:
        return 'ECB'
    else:
        return 'CBC'

In [31]:
plaintext = b'M' * 16 * 10
ciphertext = encrypt_aes_128_ecb((b'\xff' * 6) + plaintext + (b'\xf3' * 8), key=b'\x42' * 16)
detected_mode = 'ECB' if detect_ecb(plaintext,ciphertext) else 'CBC'

In [32]:
print(detected_mode)

ECB


# 2.4 Byte-at-a-time ECB decryption (Simple)

In [52]:
import base64


def encrypt(msg,key):
    key = os.urandom(16)
    add_msg = base64.b64decode(
    "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg"
    "aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq"
    "dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg"
    "YnkK"
)
    message = msg + add_msg
    result = encrypt_aes_128_ecb(message,key)
    return result

def get_block_size(msg = b"A"*1 ,key = os.urandom(16)):
    current_ctxt = None
    for i in range(2, 20):
        result_encrypt = encrypt(msg,key)
        prev_ctxt = current_ctxt or result_encrypt
        current_ctxt = result_encrypt
        if prev_ctxt[:4] == current_ctxt[:4]:
            return i-1


block_size = get_block_size()
print(block_size)

1


In [66]:
def get_payload_length(msg= b'',key= os.urandom(16)):
    result_encrypt = encrypt(msg,key)
    previous_length = len(result_encrypt)
    for i in range(20):
        encrypt_strem = encrypt(b'X'*i,key)
        length = len(encrypt_strem)
        if length != previous_length:
            result_lenght = previous_length - i
            return result_lenght
        else :
            result_lenght = previous_length
            return result_lenght

In [67]:
result_lenght = get_payload_length()
print(result_lenght)

144


In [62]:
def get_one_more_byte_ecb(known_plaintext, block_size):
    key = os.urandom(16) 
    k = len(known_plaintext)
    padding_length = (-k-1) % block_size
    padding = b"A" * padding_length
    target_block_number = len(known_plaintext) // block_size
    target_slice = slice(target_block_number*block_size,
                         (target_block_number+1)*block_size)
    target_block = encrypt(padding,key)[target_slice]

    for i in range(256):
        message = padding + known_plaintext + bytes([i])
        block = encrypt(message,key)[target_slice]
        if block == target_block:
            return bytes([i])

def get_one_byte_at_time_ecb(block_size):
    known_plaintext = b""
    payload_length = get_payload_length()
    for _ in range(payload_length):
        new_known_byte = get_one_more_byte_ecb(known_plaintext, block_size)
        known_plaintext = known_plaintext + new_known_byte
    return known_plaintext

secret = get_one_byte_at_time_ecb(block_size)

print(secret.decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



# 2.5 Byte-at-a-time ECB decryption (Harder)

In [63]:
def split_bytes_in_blocks(x, blocksize):
nb_blocks = ceil(len(x)/blocksize)
return [x[blocksize*i:blocksize*(i+1)] for i in range(nb_blocks)]
    
def encrypt_hard(message):
    key = os.urandom(16)
    prefix = os.urandom(randint(1,15))
    target = base64.b64decode(
            "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg"
            "aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq"
            "dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg"
            "YnkK"
        )
    text = prefix + message + target
    result_encrypt = encrypt_aes_128_ecb(text,key)
    return result_encrypt

In [64]:
result_encrypt = encrypt_hard(b'')
previous_length = len(result_encrypt)
for i in range(20):
    result_encrypt_stream = encrypt_hard(b'X'*i)
    length = len(result_encrypt_stream)
    if length != previous_length:
        block_size = length - previous_length
        size_prefix_plus_target_aligned = previous_length
        min_known_ptxt_size_to_align = i
        break
else:
    print('aucun changement détecté au longueur dr ciphetext ')
    

previous_blocks = None
for i in range(1, block_size+1):
    result_encrypt_stream = encrypt_hard(b'X'*i)
    blocks = split_bytes_in_blocks(result_encrypt_stream, block_size)
    if previous_blocks != None and blocks[0] == previous_blocks[0]:
        prefix_size = block_size - i + 1
        break
    previous_blocks = blocks
else:
    else(' aucun detect de block ciphertext constant')
    

target_size = size_prefix_plus_target_aligned - min_known_ptxt_size_to_align - prefix_size


key = key = os.urandom(16)
know_target_bytes = b""
for _ in range(target_size):
    r = prefix_size
    k = len(know_target_bytes)
    padding_length = (-k-1-r) % block_size
    padding = b"X" * padding_length
    target_block_number = (k+r) // block_size
    target_slice = slice(target_block_number*block_size, (target_block_number+1)*block_size)
    target_block = encrypt_hard(padding,key)[target_slice]
    for i in range(256):
        message = padding + know_target_bytes + bytes([i])
        block = encrypt_hard(message,key)[target_slice]
        if block == target_block:
            know_target_bytes += bytes([i])
            break

print(know_target_bytes.decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



# 2.6 PKCS#7 padding validation

In [69]:

block_size = 16
def pkcs7_strip(x, block_size):
    if not len(x) % block_size == 0:
        print('erreur de padding')
    last_byte = x[-1]
    padding_size = int(last_byte) 
    if not x.endswith(bytes([last_byte])*padding_size):
        print('erreur de padding')
    padding = x[:-padding_size]    
    return padding

result1 = pkcs7_strip(b'ICE ICE BABY\x04\x04\x04\x04', block_size)
print(result1)
try:
    result2 = pkcs7_strip(b'ICE ICE BABY\x05\x05\x05\x05', block_size)
    result3 = pkcs7_strip(b'ICE ICE BABY\x01\x02\x03\x04', block_size)
    print(result2)
    print(result3)
except :
    print('exception erreur de padding')


b'ICE ICE BABY'
erreur de padding
erreur de padding
b'ICE ICE BAB'
b'ICE ICE BABY'


# 2.7 CBC bitflipping attacks

In [78]:
import os
import urllib
from itertools import zip_longest
from math import ceil
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
backend = default_backend()

key = os.urandom(16)


def cbcxor(a, b, longest=True):
    if longest:
        return bytes([ x^y for (x, y) in zip_longest(a, b, fillvalue=0)])
    else:
        return bytes([ x^y for (x, y) in zip(a, b)])


def pkcs7_padding(message, block_size):
    padding_length = block_size - ( len(message) % block_size )
    if padding_length == 0:
        padding_length = block_size
    padding = bytes([padding_length]) * padding_length
    return message + padding


def pkcs7_strip(x, block_size):
    if not len(x) % block_size == 0:
        print('PaddingError')
    last_byte = x[-1]
    padding_size = int(last_byte) 
    if not x.endswith(bytes([last_byte])*padding_size):
        print('PaddingError')
    return x[:-padding_size]


def split_bytes_in_blocks(x, blocksize):
    nb_blocks = ceil(len(x)/blocksize)
    return [x[blocksize*i:blocksize*(i+1)] for i in range(nb_blocks)]


def encrypt_aes_128_block(msg, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    encryptor = cipher.encryptor()
    return encryptor.update(msg) + encryptor.finalize()

def decrypt_aes_128_block(ctxt, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    decryptor = cipher.decryptor()
    decrypted_data =  decryptor.update(ctxt) + decryptor.finalize()
    return decrypted_data    



def encrypt_aes128_cbc(msg, iv, key):
    block_size = 16
    padded_msg = pkcs7_padding(msg, block_size)
    result = b''
    mask = iv
    for block in split_bytes_in_blocks(padded_msg, block_size):
        tmp = cbcxor(block, mask)
        enc_block = encrypt_aes_128_block(tmp, key)
        mask = enc_block
        result += enc_block
    return result


def decrypt_aes128_cbc(ctxt, iv, key):
    block_size = 16
    padded_msg = b''
    mask = iv
    for enc_block in split_bytes_in_blocks(ctxt, block_size):
        tmp = decrypt_aes_128_block(enc_block, key)
        padded_msg += cbcxor(mask, tmp)
        mask = enc_block
        result = pkcs7_strip(padded_msg, block_size)
        return result


def encrypt_cbc_buffing(msg):
    quoted_msg = urllib.parse.quote_from_bytes(msg).encode()
    full_msg = (
        b"comment1=cooking%20MCs;userdata="
        + quoted_msg
        + b";comment2=%20like%20a%20pound%20of%20bacon"
    )      
    iv = os.urandom(16)
    ciphertext = encrypt_aes128_cbc(full_msg, iv, key)
    result = {"iv":iv, "ciphertext":ciphertext}
    return result

def decrypt_and_check_admin(ctxt):
    texte = decrypt_aes128_cbc(ctxt["ciphertext"], ctxt["iv"],key)     
    if b";admin=true;" in texte:
        return True
    else:
        return False

In [79]:
crypto1 = encrypt_cbc_buffing(b"hi")
print("decrypted:", decrypt_aes128_cbc(crypto1["ciphertext"], crypto1["iv"], key))
print("admin:", decrypt_and_check_admin(crypto1))

PaddingError
decrypted: b''
PaddingError
admin: False


In [107]:
crypto2 = encrypt(b"lol;admin=true")
print("decrypted:", decrypt_aes128_cbc(crypto2["ciphertext"], crypto2["iv"], key))
print("admin:", decrypt_and_check_admin(crypto2))

decrypted: b'comment1=cooking%20MCs;userdata=lol%3Badmin%3Dtrue;comment2=%20like%20a%20pound%20of%20bacon'
admin: False


In [109]:
p = cbcxor(b";admin=true", b"X"*11)
crypto3 = encrypt(b"X"*11)
crypto3["ciphertext"] = cbcxor(crypto3["ciphertext"], b'\x00'*16 + p)
print("decrypted:", decrypt_aes128_cbc(crypto3["ciphertext"], crypto3["iv"], key))
print("admin:", decrypt_and_check_admin(crypto3))

decrypted: b'comment1=cooking0\xffk\xca\x1d\x04\xf4\xb55\xe3h\xacd\xd6E\xa0;admin=true;comment2=%20like%20a%20pound%20of%20bacon'
admin: True


# 2.8 : The CBC padding oracle

In [125]:
class PaddingError(Exception):
    pass

def pkcs7_padding(message, block_size):
    padding_length = block_size - ( len(message) % block_size )
    if padding_length == 0:
        padding_length = block_size
    padding = bytes([padding_length]) * padding_length
    return message + padding

def pkcs7_strip(x, block_size):
    if not len(x) % block_size == 0:
        raise PaddingError
        
    last_byte = x[-1]
    
    # the 'int' is superfluous here
    # as last_byte is already an int (for Python a byte string is a list of integers)
    # but this way it's clearer what we are doing
    padding_size = int(last_byte)

    if padding_size > block_size:
        raise PaddingError('illegal last byte (greater than block size)')
    if padding_size == 0:
        raise PaddingError('illegal last byte (zero)')
    
    if not x.endswith(bytes([last_byte])*padding_size):
        raise PaddingError

    return x[:-padding_size]


In [126]:
import os
import random
from random import randint

messages = list(map(b64decode, [
        b'MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=',
        b'MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=',
        b'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==',
        b'MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==',
        b'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl',
        b'MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==',
        b'MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==',
        b'MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=',
        b'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=',
        b'MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93',
    ]))

key = os.urandom(16)

def encrypt():
        msg = random.choice(messages)
        iv = os.urandom(16)
        ctxt = encrypt_aes128_cbc(msg, iv, key)
        return ctxt, iv
    
def decrypt(cryptogram):
    ctxt, iv = cryptogram
    decrypt_aes128_cbc(ctxt, iv, key)
        
ctxt, iv = encrypt()
cryptogram = {'ctxt': ctxt, 'iv': iv}
 
decrypt((cryptogram['ctxt'], cryptogram['iv']))

try:
    oracle.decrypt((os.urandom(16), cryptogram['iv']))
    raise Exception('A padding error was expected')
except:
    print('Got a padding error just as expected')


Got a padding error just as expected


In [127]:
BLOCK_SIZE = 16

def cbc_xor(cryptogram, pad, index):
    ctxt = cryptogram['ctxt']
    iv = cryptogram['iv']

    if len(pad) > BLOCK_SIZE - (index % BLOCK_SIZE):
        raise ValueError('pad cannot cover several blocks')

    if isinstance(index, tuple):
        block_nb = index[0] % (len(ctxt) // BLOCK_SIZE)
        index_in_block = index[1] % (BLOCK_SIZE)

        index = block_nb*BLOCK_SIZE + index_in_block
    else:
        index = index % len(ctxt)

    if index < BLOCK_SIZE:
        iv = cbcxor(iv, (b'\x00'*index) + pad)
    else:
        ctxt = cbcxor(ctxt, b'\x00'*(index-BLOCK_SIZE) + pad)
    return {'ctxt': ctxt, 'iv':iv}


for i in range(BLOCK_SIZE):
    altered_cryptogram = cbc_xor(cryptogram, pad=b'\xff', index=len(ctxt)-1-i)
    try:
        decrypt((altered_cryptogram['ctxt'], altered_cryptogram['iv']))
        altered_cryptogram = cbc_xor(cryptogram, pad=b'\x11', index=len(ctxt)-1-i)
        decrypt((altered_cryptogram['ctxt'], altered_cryptogram['iv']))
        padding_length = i
        break
    except PaddingError:
        continue
else:
    padding_length = BLOCK_SIZE
print('padding length:', padding_length)

expected = BLOCK_SIZE - len(decrypt_aes128_cbc(ctxt, iv, key)) % BLOCK_SIZE

padding length: 16


In [128]:
known_bytes = bytes([padding_length])*padding_length

def recover_one_more_byte(cryptogram, known_bytes):
    ctxt = cryptogram['ctxt']
    iv = cryptogram['iv']
    nb_blocks_to_drop = len(known_bytes) // BLOCK_SIZE
    target_byte_index = - (len(known_bytes) % BLOCK_SIZE) - 1
    not_dropped = slice(0, -nb_blocks_to_drop*BLOCK_SIZE or None)
    l = len(known_bytes[not_dropped])
    for i in range(256):
        pad = bytes([i]) + cbcxor(known_bytes[not_dropped], bytes([l+1])*(l))
        cryptogram_to_alter = {'ctxt': ctxt[not_dropped], 'iv': iv}
        cryptogram_altered = cbc_xor(cryptogram_to_alter, pad, target_byte_index)
        try:
            decrypt((cryptogram_altered['ctxt'], cryptogram_altered['iv']))
            target_byte_value = (l+1) ^ i
            return bytes([target_byte_value])
        except PaddingError:
            continue
    else:
        err_msg = 'all values triggered a padding error'
        if l == 2:
            err_msg += '; probably a mistake when recovering last byte of the block'
        raise Exception(err_msg)



while len(known_bytes) < len(ctxt):
    new_byte = recover_one_more_byte(cryptogram, known_bytes)
    known_bytes = new_byte + known_bytes

pkcs7_strip(known_bytes, BLOCK_SIZE)

b"000008ollin' in my five point oh"